In [ ]:
%defaultDatasource jdbc:h2:mem:db

// change the paths of the csv files

In [3]:
CREATE TABLE Nutrient (
  id VARCHAR(20),
  public_id VARCHAR(60),
  name VARCHAR(80),
  PRIMARY KEY (id)
) AS SELECT
  id, public_id, name
FROM CSVREAD('filtered_foodb_nutrient.csv');

In [4]:
CREATE TABLE Compound (
    id VARCHAR(20), 
    public_id VARCHAR(20),
    name VARCHAR(100), 
    description TEXT,
    PRIMARY KEY(id)
) AS SELECT id, public_id, name, description
FROM CSVREAD('filtered_foodb_compound.csv')

In [5]:
CREATE TABLE Food (
  id VARCHAR(80),
  name VARCHAR(80),
  name_scientific VARCHAR(80),
  description text,
  food_group VARCHAR(40),
  food_subgroup VARCHAR(40),
  food_type VARCHAR(40),
  culinarydb_ingredient_id VARCHAR(40),
  foodb_id VARCHAR(50),
  PRIMARY KEY (id)
) AS SELECT
  id, name, name_scientific, description, food_group, food_subgroup, food_type, culinarydb_ingredient_id, foodb_id
FROM CSVREAD('sexto_ingredient.csv');

In [6]:
CREATE TABLE ContentCompound (
   id VARCHAR(80), 
   compound_id VARCHAR(20), 
   food_id VARCHAR(80), 
   orig_source_id VARCHAR(50), 
   orig_content DOUBLE(10), 
   orig_unit VARCHAR(50),
   standard_content VARCHAR(40),
   preparation_type VARCHAR(50),
   PRIMARY KEY(id),
   FOREIGN KEY (compound_id) REFERENCES Compound(id),
   FOREIGN KEY (food_id) REFERENCES Food(id)
) AS SELECT id, compound_id, food_id, orig_source_id, orig_content, orig_unit, standard_content, preparation_type
FROM CSVREAD('sexto_content_compound.csv')

In [ ]:
CREATE TABLE ContentNutrient (
   id VARCHAR(80), 
   nutrient_id VARCHAR(20), 
   food_id VARCHAR(80), 
   orig_source_id VARCHAR(50), 
   orig_content DOUBLE(10), 
   orig_unit VARCHAR(50),
   standard_content VARCHAR(40),
   preparation_type VARCHAR(50),
   PRIMARY KEY(id),
   FOREIGN KEY (nutrient_id) REFERENCES Nutrient(id),
   FOREIGN KEY (food_id) REFERENCES Food(id)
) AS SELECT id, nutrient_id, food_id, orig_source_id, orig_content, orig_unit, standard_content, preparation_type
FROM CSVREAD('sexto_content_nutrient.csv')

org.h2.jdbc.JdbcSQLException:  Table "CONTENTNUTRIENT" already exists; SQL statement

In [10]:
CREATE TABLE Recipe (
    id VARCHAR(20),
    title VARCHAR(200),
    cuisine VARCHAR(100),
    PRIMARY KEY(id)
) AS SELECT id, title, cuisine FROM CSVREAD('filtered_culinarydb_recipe_details.csv')

In [ ]:
CREATE TABLE RecipeFood (
    original_ingredient VARCHAR(500), 
    aliased_ingredient VARCHAR(500),
    culinarydb_ingredient_id VARCHAR(20), 
    recipe_id VARCHAR(20),
    FOREIGN KEY (recipe_id) REFERENCES Recipe(id),
    FOREIGN KEY (culinarydb_ingredient_id) REFERENCES Food(culinarydb_ingredient_id)
) AS SELECT original_ingredient, aliased_ingredient, culinarydb_ingredient_id, recipe_id
FROM CSVREAD('filtered_culinarydb_recipe_ingredients.csv')

In [15]:
-- Quais são os 5 alimentos mais ricos em um determinado nutriente?

SELECT f.name AS food, SUM(cn.orig_content) AS total_nutrient
FROM Food f
JOIN ContentNutrient cn ON f.id = cn.food_id
WHERE cn.nutrient_id = (SELECT id FROM Nutrient WHERE name = 'Carbohydrate')
GROUP BY f.name
ORDER BY total_nutrient DESC
LIMIT 5;

In [ ]:
-- Qual é a receita mais complexa (ou seja, a que tem mais ingredientes)?

SELECT r.title AS recipe, COUNT(*) AS ingredient_count
FROM Recipe r
JOIN RecipeFood rf ON r.id = rf.recipe_id
GROUP BY r.title
ORDER BY ingredient_count DESC
LIMIT 1;

In [8]:
-- Quais são os 5 compostos mais comuns encontrados nos alimentos?

SELECT c.name AS compound, COUNT(*) AS frequency
FROM Compound c
JOIN ContentCompound cc ON c.id = cc.compound_id
GROUP BY c.name
ORDER BY frequency DESC
LIMIT 5;